# Importing packages and load CNN

In [1]:
import pykat
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pykat.optics.gaussian_beams as gb
import keras
from keras import backend as k
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import tensorflow as tf
import pickle
from copy import deepcopy
from tqdm import tqdm
import time
import os

# Telling the notebook to make plots inline.
%matplotlib inline  

plt.rc('text', usetex=True)
plt.rc('font', family='serif', size=12)

n_pixl = 128
tilt_step = 1e-4
SaveModelFolder = '/home/user1/Dropbox/Academic/WORK/Beam_auto_alignment/Data/TrainedModels/'
Model_Name = 'Trained_Model_2019-07-03_17-15'

# # Read the pre-trained CNN model
# cnn = keras.models.load_model(SaveModelFolder + Model_Name + '.h5')
# # load the encoder
# loaded_Encoder = pickle.load(open(SaveModelFolder + 'Encoder_of_' + Model_Name + '.npy', 'rb'))

                                              ..-
    PyKat 1.1.331         _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,
   `^-....____:   +.      www.gwoptics.org/pykat



Using TensorFlow backend.


# Initialization

In [8]:
np.random.seed(198)
# waist size in m
waist = 140e-6
# range of movement of the waist center at the waist location in the units of waist size
Range = 3.5
# cumulative distance of waist from SM1 in m
d1 = 0.35+0.0884
# cumulative distance of waist from SM2 in m
d2 = 0.0884
dist_to_w = np.array([d1, d1, d2, d2])
# Scale for initial random misalignment in the units of waist size
a = 3.
pop_per_gen = 50
num_generations = 10
num_params = len(dist_to_w)
num_parents_mating = pop_per_gen // 10 	# 10% of new population is parents
num_offsprings_per_pair = 2 * (pop_per_gen - num_parents_mating) // num_parents_mating + 1
# after each iteration, range shrinks by
shrink_factor = 2. / pop_per_gen ** 0.25  # make sure it is < 1.
# Defining the population size.
pop_size = (pop_per_gen,num_params) # The population will have sol_per_pop chromosome \
# where each chromosome has num_weights genes.
fitness = np.empty(pop_per_gen)

## Define Functions

In [6]:
def sample_d(Rng, shape=pop_size):
	"""
	Takes i/p range in umits of beam waist at the waist location.
	Outputs sets of deltas (in radians) to be fed to steering mirrors.
	O/P shape : pop_size
	"""
	delta = np.random.uniform(low=-Rng, high=Rng, size=shape)
	delta *= waist 
	delta /= dist_to_w
	return delta

def Reward(Beam_status):
	# reward fn as total power
	base.SM1.xbeta = Beam_status[0]
	base.SM1.ybeta = Beam_status[1]
	base.SM2.xbeta = Beam_status[2]
	base.SM2.ybeta = Beam_status[3]
	out = base.run()
	Img1 = np.array(out['CCD'])
	R_fn1 = Img1.sum()/n_pixl**2/1.95749
# 	R_fn1 = np.random.random()
# 	Img1 = np.random.random()
	return R_fn1, Img1

def calc_pop_fitness(Current_beam_status, New_pop_deltas, fitness, only_offsprings=False):
	"""
	Calculating the fitness value of each solution in the current population.
	Also returns the current beam location (after adding the steps taken so far)
	"""
	if only_offsprings:
		range_vals = range(num_parents_mating, pop_per_gen)
	else:
		range_vals = range(pop_per_gen)
	for ii in range_vals:
		# take the delta step
		Current_beam_status += New_pop_deltas[ii]
		# cumulatively subtracting each delta step from all deltas
		New_pop_deltas -= New_pop_deltas[ii]
		R_new, _ = Reward(Current_beam_status)
		fitness[ii] = R_new
	return Current_beam_status, New_pop_deltas, fitness

def select_mating_pool(pop, fitness, num_parents_mating):
	"""
	Selecting the best candidates in the current generation as parents for 
	producing the offspring of the next generation.
	"""
	parents = np.empty((num_parents_mating, num_params))
	isort = np.argsort(fitness)[::-1]
	parents_fitness = fitness[isort][:num_parents_mating]
	parents = pop[isort][:num_parents_mating,:]
	return parents, parents_fitness

def get_offsprings_Uniform(pairs, parents, offspring_size):
	"""create offsprings using uniform crossover"""
	offsprings = np.empty(offspring_size)
	nn = 0
	for i in range(len(pairs)):
		for j in range(num_offsprings_per_pair):
			if nn == offspring_size[0] : break
			while True:
				# To make sure not all True/False
				i_select_genes = np.random.choice([True, False], num_params)
				if (i_select_genes.sum() != num_params) & (i_select_genes.sum() != 0): break
			offsprings[nn][i_select_genes] = parents[pairs[i][0]][i_select_genes]
			offsprings[nn][np.logical_not(i_select_genes)] = \
			parents[pairs[i][1]][np.logical_not(i_select_genes)]
			nn += 1
	return offsprings

def crossover(parents, offspring_size):
	# get all possible pairs
	pairs = []
	for p1 in range(num_parents_mating):
		for p2 in range(p1+1, num_parents_mating):
			pairs.append([p1,p2])
	pairs = np.array(pairs)
	offsprings = get_offsprings_Uniform(pairs, parents, offspring_size)
	return offsprings

def mutation(Offspring_crossover, Rng):
	# Mutation changes a single gene in each offspring randomly.
	mutations = sample_d(Rng, shape=Offspring_crossover.shape)
	# The random value to be added to the gene.
	Offspring_crossover += mutations
	return Offspring_crossover

# Base Model

In [4]:
#seed
np.random.seed(197)

base = pykat.finesse.kat()
base.verbose = False
base.parse("""

# Input laser
# -----------------------------
l laser 1e-2 0 n0            # Laser (Power = 10 mW, wavelength offset = 0)

# Gaussian Beam
gauss GB laser n0 129e-6 0   # define beam waist

s s00 0.21801 n0 nL1a         # Space (Length = 0.218 m)

# Lens of f=150mm
lens Lns 0.150 nL1a nL1b

s s0 0.032 nL1b nSM1a        # Space (Length = 0.032 m)

# Steering mirrors
# -----------------------------
bs SM1 1 0 0 45 nSM1a nSM1b nSM1c nSM1d  # Beam splitter (R=1, T=0, phi(tuning)=0, alpha=45)

s s1 0.35 nSM1b nSM2a

bs SM2 1 0 0 45 nSM2a nSM2b nSM2c nSM2d

s s2 0.0884 nSM2b nM1a

# Cavity
# -----------------------------
# cavity mirror1 (R=0.95, T=0.05, phi=0)
m M1 0.95 0.05 0 nM1a nM1b

# cavity length 122.7mm for waist 140um
s lCav 0.1227 nM1b nM2a

# cavity mirror2 (R=0.99, T=0.01, phi=0)
m M2 0.99 0.01 0 nM2a nM2b

# Setting RoC of M2
attr M2 Rc 0.150

# Defning the cavity for spatial mode basis computation
cav cav1 M1 nM1b M2 nM2a

# Output
# -----------------------------
s sOut 0.1 nM2b nOut

# Photo diode
pd P nOut

# Amplitude detector
# ad AD11 1 1 1064 nOut

# Beam camera
beam CCD nOut

# max order of modes
maxtem 4
""")

# base.parse("""
# ## Detectors ##

# # Photo diodes measureing DC-power
# pd refl nM1a          # Reflected field
# pd circ nM1b          # Circulating field
# pd tran nM2b          # Transmitted field

# ## Simulation instructions ##
# xaxis M2 phi lin -450 90 2000   # Varying tuning of input mirror m1.
# yaxis abs                       # Plotting the amplitude of the detector measurements. 
# """)
# out1 = base.run()

In [5]:
# Adding simulation instructions
base.parse("""
xaxis CCD x lin -15 15 63
x2axis CCD y lin -15 15 63
yaxis abs
""")

## Initial misalignment

In [9]:
# give angle in radian
current_beam_status = np.random.uniform(low=-a, high=a, size=(num_params,))
current_beam_status *= waist 
current_beam_status /= dist_to_w
print(current_beam_status)

[ 0.00021039  0.0008793  -0.00182743 -0.00417707]


In [10]:
#Creating the initial population of deltas
new_pop_deltas = sample_d(Range, shape=pop_size)
print("\n\nRange:", Range, "x waist")

for gen in range(num_generations):
	# Shrink range
	Range *= shrink_factor
	print("\n\n Gen: {} Range: {} x waist".format(gen, Range))
	if gen == 0:
		current_beam_status, new_pop_deltas, fitness = calc_pop_fitness(current_beam_status, \
		new_pop_deltas, fitness, only_offsprings=False)
	else:
		current_beam_status, new_pop_deltas, fitness = calc_pop_fitness(current_beam_status, \
		new_pop_deltas, fitness, only_offsprings=True)
	print(fitness)
	# Selecting the best parents in the population for mating.
	parents, parents_fitness = select_mating_pool(new_pop_deltas, fitness, num_parents_mating)
	# Generating next generation using crossover.
	offspring_crossover = crossover(parents, (pop_per_gen - num_parents_mating, num_params))
	# Adding some variations to the offsrping using mutation.
	offspring_mutation = mutation(offspring_crossover, Range)
	# Creating the new population based on the parents and offspring.
	new_pop_deltas[0:num_parents_mating, :] = parents
	new_pop_deltas[num_parents_mating:, :] = offspring_mutation
	fitness[0:num_parents_mating] = parents_fitness
	fitness[num_parents_mating:] *= 0.

##########Range: 3.5 x waist
##########Range: 2.6324221651604756 x waist
[1.37169865e-10 1.48408936e-22 3.52032312e-23 6.24537670e-45
 3.84248001e-27 4.50287016e-24 5.87666673e-42 3.04245981e-21
 2.22419039e-42 1.17218600e-46 1.38847329e-28 1.76881631e-15
 1.68879331e-03 1.99495761e-43 1.07862113e-03 2.24096601e-14
 3.94309451e-43 5.49399498e-63 2.86798788e-30 9.56975601e-62
 3.45055221e-52 1.79848882e-02 4.37202370e-25 1.25721987e-44
 1.64878128e-19 3.68205804e-46 1.45385494e-36 4.06365810e-42
 1.44116870e-78 3.19362901e-34 1.26521551e-09 1.00521864e-20
 9.72058457e-45 4.58653051e-08 2.01705111e-14 6.30360207e-20
 4.24169546e-14 1.03721169e-96 5.37477361e-19 7.24672395e-66
 5.19982911e-33 1.74242422e-18 5.46047513e-08 1.52267844e-09
 3.36843690e-14 1.64015098e-49 1.13703532e-76 3.79296731e-05
 2.62051396e-44 4.95857808e-39]
##########Range: 1.9798989873223332 x waist
[1.79848882e-02 1.68879331e-03 1.07862113e-03 3.79296731e-05
 5.46047513e-08 7.88834906e-19 5.54433914e-12 2.90896906e-